In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cmocean as cmo
import datetime
import xarray as xr
import os
import matplotlib.colors as mcolors
import cmocean as cmo
import pandas as pd
import matplotlib.dates as mdates
import geopandas as gpd

import process_modis as process

In [2]:
plt.style.use('ccic_climate_record_analysis/ccic.mplstyle')


Bad key axes3d.xaxis.panecolor in file ccic_climate_record_analysis/ccic.mplstyle, line 306 ('axes3d.xaxis.panecolor:    (0.95, 0.95, 0.95, 0.5)  # background pane on 3D axes')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.6.2/matplotlibrc.template
or from the matplotlib source distribution

Bad key axes3d.yaxis.panecolor in file ccic_climate_record_analysis/ccic.mplstyle, line 307 ('axes3d.yaxis.panecolor:    (0.90, 0.90, 0.90, 0.5)  # background pane on 3D axes')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.6.2/matplotlibrc.template
or from the matplotlib source distribution

Bad key axes3d.zaxis.panecolor in file ccic_climate_record_analysis/ccic.mplstyle, line 308 ('axes3d.zaxis.panecolor:    (0.925, 0.925, 0.925, 0.5)  # background pane on 3D axes')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.6

# Interpolation/masking

Just a sanity check to see if all looks okay for a single day of data. Actual interpolation and masking done during time series calculation.

In [4]:
file_path = './data/modis/ccic_modis_data_2020.nc'
dataset = xr.open_dataset(file_path)

In [5]:
latitude = dataset['lat'][:]
longitude = dataset['lon'][:]
data = dataset['Cloud_Fraction_Mean'][20,:,:].values
data_filtered = np.where((data == -9999) | (data == -2147483647), np.nan, data)
data_filtered = data_filtered.astype(float) * 0.0001

In [ ]:
data_interpolated, data_masked, _, _ = process.mask_data(data_filtered, dataset["lat"][:], dataset["lon"][:])

In [ ]:
fig, axes = plt.subplots(2, 2, figsize = (10,4))

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

extent = [longitude[0]-0.5, longitude[-1]+0.5, latitude[0]-0.5, latitude[-1]+0.5]

cax = axes[0,0].imshow(data_filtered, extent=extent)
cbar = fig.colorbar(cax)
cbar.set_label(r'Cloud fraction')
world.boundary.plot(ax=axes[0,0], color='black', lw=1)
axes[0, 0].set_title(r'MODIS L3')

axes[0,1].axis('off')

cax1 = axes[1,0].imshow(data_interpolated, extent=extent)
cbar1 = fig.colorbar(cax1)
cbar1.set_label(r'Cloud fraction')
world.boundary.plot(ax=axes[1,0], color='black', lw=1)
axes[1, 0].set_title(r'Interpolated')

cax2 = axes[1,1].imshow(data_masked, extent=extent)
world.boundary.plot(ax=axes[1,1], color='black', lw=1)
cbar2 = fig.colorbar(cax2)
cbar2.set_label(r'Cloud fraction')
axes[1, 1].set_title(r'Masked')

fig.tight_layout()


# Time series

In [6]:
CF_global_mean = {}

directory_path = './data/modis/'

for file in os.listdir(directory_path):
    if file.endswith('.nc'):
        file_path = os.path.join(directory_path, file)

        dataset = xr.open_dataset(file_path)

        latitude = dataset["lat"][:]
        longitude = dataset["lon"][:]
        
        dates_unix = dataset["date"][:]
        dates_unix = np.where(
        dates_unix == -9223372036854775806, 0, dates_unix
        )
        year = datetime.datetime.fromtimestamp(dates_unix[-1]).year
        print(year)

        CF_global_mean.update({f'{year}': process.modis_monthly(dataset, latitude, longitude, 'Cloud_Fraction_Mean')})


2019


KeyboardInterrupt: 

In [ ]:
data_points = []
for year in CF_global_mean:
    for date in CF_global_mean[year]:
        date_obj = datetime.datetime.strptime(f"{year}-{int(float(date))}", "%Y-%m")
        data_points.append((date_obj, CF_global_mean[year][date]))

sorted_data_points = sorted(data_points, key=lambda x: x[0])

dates, values = zip(*sorted_data_points)

fig, ax = plt.subplots()

ax.plot(dates, values)

ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
ax.set_xticks(dates[::12])
ax.set_xlabel('Time')
ax.set_ylabel('CF global mean [-]')
ax.grid()
